# 04 - Симметричное шифрование: AES и режимы работы

Этот notebook покрывает уроки CRYPTO-06 (AES) и CRYPTO-07 (режимы блочных шифров).

**Что вы изучите:**
1. Базовые операции AES (ECB — только для обучения)
2. Режимы шифрования: CBC, CTR, GCM
3. Практический пример: шифрование с аутентификацией
4. Уязвимости: ECB паттерны, повторное использование nonce

**Используемая библиотека:** `pycryptodome` (Crypto.Cipher.AES)

In [ ]:
# Импорты
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import os

print("pycryptodome загружен успешно")

---
## Часть 1: AES базовые операции (CRYPTO-06)

AES шифрует блоки по 16 байт (128 бит). Ключ может быть 16, 24 или 32 байта (AES-128/192/256).

In [ ]:
# AES-ECB: базовое шифрование одного блока
# ВНИМАНИЕ: ECB небезопасен для реальных данных! Только для обучения.

key = get_random_bytes(32)  # AES-256: 32 байта = 256 бит
cipher = AES.new(key, AES.MODE_ECB)

# Открытый текст должен быть ровно 16 байт
plaintext = b"Exactly16Bytes!!"
assert len(plaintext) == 16, "Блок должен быть 16 байт"

ciphertext = cipher.encrypt(plaintext)
print(f"Ключ (hex):       {key.hex()}")
print(f"Открытый текст:   {plaintext}")
print(f"Шифротекст (hex): {ciphertext.hex()}")

# Расшифровка
decipher = AES.new(key, AES.MODE_ECB)
decrypted = decipher.decrypt(ciphertext)
print(f"Расшифровано:     {decrypted}")
assert decrypted == plaintext, "Ошибка расшифровки!"
print("\nУспех: расшифрованный текст совпадает с оригиналом")

In [ ]:
# Демонстрация проблемы ECB: одинаковые блоки -> одинаковый шифротекст

key = get_random_bytes(16)  # AES-128
cipher = AES.new(key, AES.MODE_ECB)

# Два одинаковых блока
block_a = b"AAAAAAAAAAAAAAAA"  # 16 байт 'A'
block_b = b"AAAAAAAAAAAAAAAA"  # 16 байт 'A' (тот же)
block_c = b"BBBBBBBBBBBBBBBB"  # 16 байт 'B' (другой)

ct_a = cipher.encrypt(block_a)
ct_b = cipher.encrypt(block_b)
ct_c = cipher.encrypt(block_c)

print("ECB: одинаковые блоки -> одинаковый шифротекст")
print(f"Block A: {ct_a.hex()}")
print(f"Block B: {ct_b.hex()}")
print(f"Block C: {ct_c.hex()}")
print(f"\nA == B? {ct_a == ct_b}  <-- Проблема! Видны одинаковые блоки")
print(f"A == C? {ct_a == ct_c}  <-- Разные блоки -> разный шифротекст")

In [ ]:
# Размеры ключей AES
for key_size in [16, 24, 32]:  # AES-128, AES-192, AES-256
    key = get_random_bytes(key_size)
    cipher = AES.new(key, AES.MODE_ECB)
    ct = cipher.encrypt(b"Test block 128b!")
    
    bits = key_size * 8
    rounds = {128: 10, 192: 12, 256: 14}[bits]
    print(f"AES-{bits}: ключ={key_size} байт, раундов={rounds}, шифротекст={ct.hex()[:16]}...")

---
## Часть 2: Режимы шифрования (CRYPTO-07)

ECB опасен. Вот безопасные режимы: CBC, CTR, GCM.

### CBC (Cipher Block Chaining)

Каждый блок XOR-ится с предыдущим шифротекстом перед шифрованием.
Требует IV (вектор инициализации) и паддинг.

In [ ]:
# CBC режим
key = get_random_bytes(16)
data = b"Hello, blockchain world! This message is longer than one block."

# Шифрование
cipher_enc = AES.new(key, AES.MODE_CBC)
ct = cipher_enc.encrypt(pad(data, AES.block_size))  # PKCS7 паддинг
iv = cipher_enc.iv

print(f"IV (случайный):  {iv.hex()}")
print(f"Шифротекст:      {ct.hex()[:48]}...")
print(f"Размер данных:   {len(data)} байт")
print(f"Размер шифротекста: {len(ct)} байт (с паддингом до кратного 16)")

# Расшифровка
cipher_dec = AES.new(key, AES.MODE_CBC, iv=iv)
pt = unpad(cipher_dec.decrypt(ct), AES.block_size)
assert pt == data
print(f"\nРасшифровано: {pt.decode()}")

In [ ]:
# CBC: одинаковые блоки дают РАЗНЫЙ шифротекст (в отличие от ECB)
key = get_random_bytes(16)

# Два одинаковых сообщения, но с разными IV
msg = b"AAAAAAAAAAAAAAAA" * 3  # 48 байт одинаковых блоков

cipher1 = AES.new(key, AES.MODE_CBC)
ct1 = cipher1.encrypt(msg)

cipher2 = AES.new(key, AES.MODE_CBC)
ct2 = cipher2.encrypt(msg)

print("CBC: одинаковые данные + разные IV = разные шифротексты")
print(f"CT1: {ct1.hex()[:48]}...")
print(f"CT2: {ct2.hex()[:48]}...")
print(f"Равны? {ct1 == ct2}  <-- Нет! CBC скрывает паттерны")

# Внутри одного шифротекста: блоки тоже разные
blocks = [ct1[i:i+16] for i in range(0, len(ct1), 16)]
print(f"\nБлоки внутри CT1:")
for i, b in enumerate(blocks):
    print(f"  Block {i}: {b.hex()}")
print(f"Block 0 == Block 1? {blocks[0] == blocks[1]}  <-- Разные!")

### CTR (Counter Mode)

Превращает блочный шифр в потоковый. Шифрует счетчик, XOR-ит с данными.
Не нужен паддинг. Можно параллелизировать.

In [ ]:
# CTR режим
key = get_random_bytes(16)
data = b"Ethereum keystore uses AES-128-CTR for encrypting private keys"

# Шифрование
cipher_enc = AES.new(key, AES.MODE_CTR)
ct = cipher_enc.encrypt(data)  # Без паддинга!
nonce = cipher_enc.nonce

print(f"Nonce:         {nonce.hex()}")
print(f"Шифротекст:    {ct.hex()[:48]}...")
print(f"Размер данных: {len(data)} байт")
print(f"Размер CT:     {len(ct)} байт  <-- Тот же размер! Нет паддинга")

# Расшифровка
cipher_dec = AES.new(key, AES.MODE_CTR, nonce=nonce)
pt = cipher_dec.decrypt(ct)
assert pt == data
print(f"\nРасшифровано: {pt.decode()}")

### GCM (Galois/Counter Mode)

CTR + аутентификация. Стандарт для TLS 1.3 и современных протоколов.
Обеспечивает конфиденциальность И целостность данных.

In [ ]:
# GCM режим: аутентифицированное шифрование
key = get_random_bytes(16)
data = b"Transfer 1.5 ETH to 0xAbCdEf1234567890"
header = b"tx_version=2"  # AAD: аутентифицируется, но НЕ шифруется

# Шифрование
cipher_enc = AES.new(key, AES.MODE_GCM)
cipher_enc.update(header)  # Добавляем Associated Authenticated Data
ct, tag = cipher_enc.encrypt_and_digest(data)
nonce = cipher_enc.nonce

print(f"Nonce:      {nonce.hex()}")
print(f"Шифротекст: {ct.hex()[:48]}...")
print(f"Auth Tag:   {tag.hex()}")
print(f"AAD:        {header.decode()} (не зашифрован, но защищен тегом)")

# Расшифровка с проверкой целостности
cipher_dec = AES.new(key, AES.MODE_GCM, nonce=nonce)
cipher_dec.update(header)
pt = cipher_dec.decrypt_and_verify(ct, tag)
print(f"\nРасшифровано: {pt.decode()}")
print("Целостность подтверждена!")

In [ ]:
# GCM: обнаружение подделки
# Что произойдет, если злоумышленник изменит шифротекст?

tampered_ct = bytearray(ct)
tampered_ct[0] ^= 0x01  # Изменяем один бит

cipher_check = AES.new(key, AES.MODE_GCM, nonce=nonce)
cipher_check.update(header)

try:
    cipher_check.decrypt_and_verify(bytes(tampered_ct), tag)
    print("Тег совпал — подделка не обнаружена")
except ValueError as e:
    print(f"ПОДДЕЛКА ОБНАРУЖЕНА: {e}")
    print("GCM обнаружил изменение даже одного бита!")
    print("Без GCM злоумышленник мог бы незаметно изменить данные.")

In [ ]:
# Сравнение режимов: один и тот же текст в разных режимах

key = get_random_bytes(16)
data = b"Same plaintext!!" * 3  # 48 байт (3 одинаковых блока)

# ECB
ecb = AES.new(key, AES.MODE_ECB)
ct_ecb = ecb.encrypt(data)

# CBC
cbc = AES.new(key, AES.MODE_CBC)
ct_cbc = cbc.encrypt(data)

# CTR
ctr = AES.new(key, AES.MODE_CTR)
ct_ctr = ctr.encrypt(data)

print("Один и тот же текст (3 одинаковых блока) в разных режимах:")
print(f"\nECB: {ct_ecb.hex()}")
print(f"     Block 0: {ct_ecb[:16].hex()}")
print(f"     Block 1: {ct_ecb[16:32].hex()}")
print(f"     Block 2: {ct_ecb[32:48].hex()}")
print(f"     Блоки одинаковы? {ct_ecb[:16] == ct_ecb[16:32]}  <-- ПРОБЛЕМА ECB!")

print(f"\nCBC: {ct_cbc.hex()}")
print(f"     Block 0: {ct_cbc[:16].hex()}")
print(f"     Block 1: {ct_cbc[16:32].hex()}")
print(f"     Block 2: {ct_cbc[32:48].hex()}")
print(f"     Блоки одинаковы? {ct_cbc[:16] == ct_cbc[16:32]}  <-- Нет!")

print(f"\nCTR: {ct_ctr.hex()}")
print(f"     Размер: {len(ct_ctr)} байт (нет паддинга)")

---
## Часть 3: Практический пример

Реальный паттерн шифрования: GCM для безопасной передачи данных.

In [ ]:
# Практический паттерн: безопасное шифрование сообщения с GCM

import json
import base64

def encrypt_message(key: bytes, plaintext: str, associated_data: str = "") -> dict:
    """Зашифровать сообщение с AES-GCM (production pattern)."""
    cipher = AES.new(key, AES.MODE_GCM)
    if associated_data:
        cipher.update(associated_data.encode())
    ct, tag = cipher.encrypt_and_digest(plaintext.encode('utf-8'))
    
    return {
        'nonce': base64.b64encode(cipher.nonce).decode(),
        'ciphertext': base64.b64encode(ct).decode(),
        'tag': base64.b64encode(tag).decode(),
        'aad': associated_data,
    }

def decrypt_message(key: bytes, encrypted: dict) -> str:
    """Расшифровать сообщение с проверкой целостности."""
    nonce = base64.b64decode(encrypted['nonce'])
    ct = base64.b64decode(encrypted['ciphertext'])
    tag = base64.b64decode(encrypted['tag'])
    
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    if encrypted.get('aad'):
        cipher.update(encrypted['aad'].encode())
    
    pt = cipher.decrypt_and_verify(ct, tag)
    return pt.decode('utf-8')

# Использование
key = get_random_bytes(32)  # AES-256
message = "Приватный ключ кошелька: 0x1234...abcd"

encrypted = encrypt_message(key, message, associated_data="wallet_v2")
print("Зашифрованное сообщение:")
print(json.dumps(encrypted, indent=2))

decrypted = decrypt_message(key, encrypted)
print(f"\nРасшифровано: {decrypted}")
assert decrypted == message

In [ ]:
# Обнаружение подделки в практическом примере

# Попробуем изменить шифротекст
tampered = encrypted.copy()
ct_bytes = bytearray(base64.b64decode(tampered['ciphertext']))
ct_bytes[0] ^= 0xFF  # Изменяем первый байт
tampered['ciphertext'] = base64.b64encode(bytes(ct_bytes)).decode()

try:
    decrypt_message(key, tampered)
    print("Подделка не обнаружена (это было бы плохо)")
except ValueError:
    print("Подделка обнаружена! Данные отклонены.")
    print("GCM гарантирует: если данные изменены, расшифровка завершится ошибкой.")

In [ ]:
# Простая утилита шифрования файлов

def encrypt_file(key: bytes, input_path: str, output_path: str) -> dict:
    """Зашифровать файл с AES-256-GCM."""
    with open(input_path, 'rb') as f:
        data = f.read()
    
    cipher = AES.new(key, AES.MODE_GCM)
    cipher.update(input_path.encode())  # AAD: имя файла
    ct, tag = cipher.encrypt_and_digest(data)
    
    with open(output_path, 'wb') as f:
        f.write(cipher.nonce)  # 16 байт
        f.write(tag)           # 16 байт
        f.write(ct)            # шифротекст
    
    return {'nonce': cipher.nonce.hex(), 'tag': tag.hex(), 'size': len(ct)}

def decrypt_file(key: bytes, input_path: str, output_path: str, original_name: str) -> bool:
    """Расшифровать файл с проверкой целостности."""
    with open(input_path, 'rb') as f:
        nonce = f.read(16)
        tag = f.read(16)
        ct = f.read()
    
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    cipher.update(original_name.encode())
    
    try:
        pt = cipher.decrypt_and_verify(ct, tag)
        with open(output_path, 'wb') as f:
            f.write(pt)
        return True
    except ValueError:
        print("Ошибка: файл поврежден или подделан!")
        return False

# Демонстрация
key = get_random_bytes(32)

# Создаем тестовый файл
test_content = b"Secret data: private key 0x" + os.urandom(32).hex().encode()
with open('/tmp/test_plain.dat', 'wb') as f:
    f.write(test_content)

# Шифруем
info = encrypt_file(key, '/tmp/test_plain.dat', '/tmp/test_encrypted.dat')
print(f"Зашифровано: {info}")

# Расшифровываем
ok = decrypt_file(key, '/tmp/test_encrypted.dat', '/tmp/test_decrypted.dat', '/tmp/test_plain.dat')

with open('/tmp/test_decrypted.dat', 'rb') as f:
    result = f.read()

assert result == test_content
print(f"Файл успешно расшифрован! Размер: {len(result)} байт")

# Очистка
for p in ['/tmp/test_plain.dat', '/tmp/test_encrypted.dat', '/tmp/test_decrypted.dat']:
    if os.path.exists(p):
        os.remove(p)

---
## Упражнения

Выполните каждое упражнение в ячейке ниже.

### Упражнение 1: Сравнение режимов

Зашифруйте строку `b"AAAAAAAAAAAAAAAA" * 4` (64 байта одинаковых блоков) в режимах ECB, CBC и CTR.
Сравните выходы: в каком режиме видны повторяющиеся паттерны?

In [ ]:
# Упражнение 1: ваш код здесь
key = get_random_bytes(16)
data = b"AAAAAAAAAAAAAAAA" * 4

# ECB
# ...

# CBC
# ...

# CTR
# ...

### Упражнение 2: Уязвимость повторного nonce в CTR

Зашифруйте два РАЗНЫХ сообщения одним ключом и одним nonce в режиме CTR.
Вычислите XOR двух шифротекстов. Покажите, что результат равен XOR двух открытых текстов.

**Подсказка:** `bytes(a ^ b for a, b in zip(ct1, ct2))` для XOR.

In [ ]:
# Упражнение 2: ваш код здесь
key = get_random_bytes(16)

msg1 = b"Hello, World!!!!"  # 16 байт
msg2 = b"Attack at dawn!!"  # 16 байт

# Шифруем с одним nonce (ОПАСНО!)
# nonce = get_random_bytes(8)
# cipher1 = AES.new(key, AES.MODE_CTR, nonce=nonce)
# cipher2 = AES.new(key, AES.MODE_CTR, nonce=nonce)  # Тот же nonce!

# ct1 = cipher1.encrypt(msg1)
# ct2 = cipher2.encrypt(msg2)

# xor_ct = bytes(a ^ b for a, b in zip(ct1, ct2))
# xor_pt = bytes(a ^ b for a, b in zip(msg1, msg2))

# print(f"XOR шифротекстов: {xor_ct.hex()}")
# print(f"XOR открытых текстов: {xor_pt.hex()}")
# print(f"Равны? {xor_ct == xor_pt}")

### Упражнение 3: Зашифрованный обмен сообщениями

Реализуйте функции `send_message(key, msg)` и `receive_message(key, packet)`
с использованием AES-GCM. Формат пакета: `nonce (16 байт) + tag (16 байт) + ciphertext`.

In [ ]:
# Упражнение 3: ваш код здесь

def send_message(key: bytes, msg: str) -> bytes:
    """Зашифровать сообщение, вернуть пакет."""
    # Ваш код...
    pass

def receive_message(key: bytes, packet: bytes) -> str:
    """Расшифровать пакет, вернуть сообщение."""
    # Ваш код...
    pass

# Тест:
# key = get_random_bytes(32)
# packet = send_message(key, "Привет, блокчейн!")
# result = receive_message(key, packet)
# assert result == "Привет, блокчейн!"
# print(f"Сообщение доставлено: {result}")

---
## Челлендж: Padding Oracle Attack (концепция)

Padding Oracle Attack — это атака на CBC режим, где сервер по-разному реагирует
на ошибку паддинга и ошибку расшифровки.

**Задача:** Объясните (в markdown ячейке ниже), как злоумышленник может использовать
различие в ответах сервера для расшифровки данных байт за байтом.

**Подсказка:** В PKCS7 последний блок содержит паддинг (например, `\x03\x03\x03`).
Если злоумышленник изменяет предпоследний блок шифротекста, он влияет на расшифровку
последнего блока через XOR в CBC.

**Ваш ответ:**

(Опишите концепцию Padding Oracle Attack здесь)

1. ...
2. ...
3. ...

In [ ]:
# Бонус: демонстрация различия ответов сервера (padding oracle concept)

def vulnerable_server(key: bytes, iv: bytes, ciphertext: bytes) -> str:
    """Уязвимый сервер: раскрывает ошибку паддинга."""
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    pt = cipher.decrypt(ciphertext)
    
    # Проверяем паддинг вручную (PKCS7)
    pad_byte = pt[-1]
    if pad_byte > 16 or pad_byte == 0:
        return "PADDING_ERROR"  # <-- Утечка информации!
    
    if pt[-pad_byte:] != bytes([pad_byte]) * pad_byte:
        return "PADDING_ERROR"  # <-- Утечка информации!
    
    return "OK"

# Демонстрация: сервер дает разные ответы
key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_CBC)
ct = cipher.encrypt(pad(b"Secret!", 16))
iv = cipher.iv

print(f"Нормальный запрос:    {vulnerable_server(key, iv, ct)}")

# Изменяем последний байт IV (влияет на расшифровку первого блока)
bad_iv = bytearray(iv)
bad_iv[-1] ^= 0x01
print(f"Измененный IV:        {vulnerable_server(key, bytes(bad_iv), ct)}")
print("\n--> Различие в ответах позволяет атакующему постепенно")
print("    восстановить открытый текст без знания ключа!")
print("    Вот почему GCM лучше CBC: он обнаруживает любые изменения.")